In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime, timedelta
from tqdm import tqdm
from time import sleep
from processor.processor import Processor as p

In [2]:
market = Market()
market.connect()
sp5 = market.retrieve("sp100")
market.disconnect()

In [3]:
start = (datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")

In [5]:
analysis = []
market.connect()
periods = ["year","quarter","week"]
for ticker in tqdm(sp5["ticker"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("stocks",ticker)
        test = p.column_date_processing(test)
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["rolling_100"] = test["adjopen"].rolling(window=100).mean()
        test["window_10"] = test["adjopen"].shift(10)
        ep = test.tail(1)["adjopen"].item()
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjopen"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjopen"].item())/test.tail(2).iloc[0]["adjopen"].item()
        report["window_gain"] = (test.tail(2).iloc[0]["window_10"].item() - ep)/ep
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 503/503 [00:08<00:00, 57.08it/s]


In [7]:
period = "day"
a = pd.DataFrame(analysis)
a = a.merge(sp5[["ticker","Security","GICS Sector"]],on="ticker")
a.sort_values(f"{period}_gain",ascending=False).head(20)

,year_gain,quarter_gain,week_gain,day_gain,window_gain,ticker,Security,GICS Sector
408,0.024950,0.068298,0.040308,0.050062,-0.051634,ROST,Ross Stores,Consumer Discretionary
45,0.474827,-0.019317,0.005427,0.036849,0.036100,AMAT,Applied Materials,Information Technology
216,0.160065,-0.247344,0.009890,0.026070,-0.022761,GNRC,Generac,Industrials
176,-0.277684,0.056536,-0.014856,0.024347,0.023518,EPAM,EPAM Systems,Information Technology
57,0.189372,0.028369,-0.031556,0.024106,-0.114943,AXON,Axon Enterprise,Industrials
101,0.134643,0.063480,-0.054285,0.022990,0.024048,CDAY,Ceridian,Information Technology
499,0.045273,-0.081986,0.030774,0.022366,-0.082469,ZBRA,Zebra Technologies,Information Technology
110,-0.141440,-0.015410,-0.047716,0.020257,0.042136,CI,Cigna,Health Care
137,-0.000324,-0.101048,0.001622,0.019822,0.066084,CSX,CSX,Industrials
399,0.127597,0.131335,0.004973,0.019668,-0.050406,REGN,Regeneron,Health Care


In [8]:
a.groupby("GICS Sector").mean().sort_values("quarter_gain",ascending=False).head(20)

,year_gain,quarter_gain,week_gain,day_gain,window_gain
GICS Sector,,,,,
Energy,0.093254,0.116606,-0.009196,0.008077,-0.017673
Financials,-0.030690,0.004330,-0.027887,-0.001599,0.044250
Industrials,0.113498,-0.022299,-0.026516,0.002397,0.029019
Communication Services,0.171445,-0.022537,-0.026171,0.000204,0.013070
Materials,0.005029,-0.026181,-0.026771,-0.000325,0.054471
Health Care,-0.007884,-0.026222,-0.031681,-0.002609,0.032546
Consumer Staples,-0.038291,-0.035832,-0.021667,0.004142,0.043208
Utilities,-0.075435,-0.039085,-0.011851,0.004461,0.028870
Real Estate,-0.015014,-0.040702,-0.025651,0.002414,0.041141


In [9]:
a = pd.DataFrame(analysis)
a.sort_values("window_gain",ascending=False).head(10)

,year_gain,quarter_gain,week_gain,day_gain,window_gain,ticker
402,-0.207738,-0.228185,-0.083937,0.005279,0.327639,RMD
206,0.178071,-0.234396,-0.041904,-0.004355,0.325402,FTNT
253,-0.283801,-0.276347,-0.083903,-0.015291,0.293386,PODD
415,-0.314502,-0.146819,-0.080576,-0.025728,0.286999,SEE
256,-0.397103,-0.201179,-0.019701,0.013849,0.258596,IFF
438,-0.109194,-0.205376,-0.027004,-0.000289,0.243134,TPR
275,-0.243185,-0.225237,-0.176197,-0.137821,0.225709,KEYS
445,0.993432,-0.229898,-0.101226,-0.017015,0.203397,TSLA
423,-0.431244,-0.399675,-0.082773,-0.002642,0.169090,SEDG
148,-0.075656,-0.165092,-0.083880,-0.016325,0.151061,DXCM
